<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/leccion_1_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lección 1.3.5: BigQuery y Google Sheets: Análisis de Datos en la Nube

## 1. ¿Por qué esta combinación importa?

En el mundo real de datos, **no se trata de elegir herramientas, sino de combinarlas**. Google Sheets y BigQuery son dos caras de la misma moneda:

- **Google Sheets**: Agilidad para análisis colaborativo y prototipado rápido
- **BigQuery**: Potencia industrial para datasets masivos
- **Juntas**: Flujo continuo desde la recolección hasta el insight

> **Analogía:** Sheets es tu taller de diseño ágil, BigQuery es tu fábrica a escala industrial. Ambos son necesarios.

**Ejemplo real:**
Un equipo de marketing usa Sheets para recoger datos de campañas vía APIs, los procesa en BigQuery para análisis cross-channel, y vuelve a Sheets para dashboards ejecutivos. **Todo integrado.**

---

## 2. Google Sheets: más que una hoja de cálculo

Sheets ha evolucionado de herramienta de cálculo a **plataforma colaborativa de análisis en la nube**.

**Ventajas clave:**
- ✅ Colaboración en tiempo real con historial de cambios
- ✅ Integración nativa con BigQuery, Data Studio, Apps Script
- ✅ Automatización sin código (macros, triggers)
- ✅ Conectores directos a APIs y fuentes externas

**Funciones avanzadas:**
```excel
=QUERY(A1:G1000, "SELECT A, AVG(B) WHERE C = 'Premium' GROUP BY A")
=IMPORTXML("https://example.com", "//div[@class='price']")
=GOOGLEFINANCE("NASDAQ:GOOGL", "price", TODAY()-30, TODAY())
```

**Caso práctico:**
Un equipo de ventas usa Google Forms conectado a Sheets para capturar leads, aplica filtros colaborativos, crea gráficos en tiempo real, y exporta automáticamente a BigQuery para análisis histórico.

---

## 3. BigQuery: el motor de datos enterprise

BigQuery es un **data warehouse serverless** que redefine lo que significa escalar: procesa terabytes en segundos sin gestión de infraestructura.

**Lo que lo hace único:**
- **Totalmente serverless**: Cero configuración, cero servidores
- **Pago por uso**: Solo pagas por los datos que procesas
- **SQL estándar**: Tu conocimiento funciona inmediatamente
- **ML integrado**: Crea modelos con SQL

**Ejemplo de consulta real:**
```sql
-- Análisis de cohorte de usuarios
WITH user_cohorts AS (
  SELECT
    user_id,
    DATE_TRUNC(DATE(signup_date), MONTH) as signup_month,
    DATE_DIFF(DATE(event_date), DATE(signup_date), MONTH) as months_active
  FROM `project.analytics.user_events`
  WHERE event_name = 'purchase'
)
SELECT
  signup_month,
  months_active,
  COUNT(DISTINCT user_id) as active_users,
  ROUND(COUNT(DISTINCT user_id) / MAX(COUNT(DISTINCT user_id))
        OVER(PARTITION BY signup_month), 2) as retention_rate
FROM user_cohorts
GROUP BY 1, 2
ORDER BY 1, 2;
```

**Impacto empresarial:** Una fintech analiza 100M de transacciones para detectar fraude en **8 segundos** vs 4 horas en sistemas tradicionales.

---

## 4. Cuándo usar cada herramienta

| Aspecto | Google Sheets | BigQuery |
|---------|---------------|----------|
| **Volumen óptimo** | Hasta ~100K filas | Millones a petabytes |
| **Colaboración** | ⭐⭐⭐ Excelente | ⭐ Limitada |
| **Modelo costo** | Fijo (Workspace) | Por uso (almacenamiento + consultas) |
| **Velocidad** | Segundos-minutos | Segundos en cualquier volumen |
| **Análisis complejo** | Limitado (fórmulas) | Ilimitado (SQL, ML, UDFs) |
| **Mejor para** | Prototipado, dashboards ligeros, datos operativos | ETL, análisis histórico, ML |

> **Regla práctica:** Usa Sheets hasta ~100K filas o cuando la colaboración es clave. Salta a BigQuery cuando necesites velocidad, escala o análisis avanzado.

---

## 5. La integración bidireccional

La verdadera magia surge cuando **conectas ambos mundos**.

### De Sheets → BigQuery: Escalando el análisis
**Caso típico:** Datos de encuestas o presupuestos que necesitan análisis avanzado.

```sql
-- En BigQuery, accediendo a Sheets como tabla externa
SELECT
  department,
  AVG(budget_utilization) as avg_utilization
FROM `my-project.analytics.budget_tracking_sheets`
WHERE quarter = 'Q1-2024'
GROUP BY department;
```

### De BigQuery → Sheets: Democratizando insights
**Caso típico:** Resultados de análisis complejos para stakeholders no técnicos.

```excel
=QUERY(IMPORTDATA("bigquery://my-project/sales_performance"),
       "SELECT region, sales_rep, revenue WHERE revenue > 50000")
```

**Flujo de trabajo completo:**
```
Datos (Forms/APIs)
  → Sheets (recolección)
  → BigQuery (procesamiento)
  → Sheets (visualización)
```

---

## 6. Caso de estudio: Startup de e-commerce

**Problema inicial:**
- Crecimiento 10x en 6 meses satura el análisis en Sheets
- 5 hojas diferentes sin consolidar
- 4 horas diarias de consolidación manual
- Imposible analizar tendencias históricas

**Evolución por fases:**

| Fase | Herramienta | Límite encontrado |
|------|-------------|-------------------|
| **1-3 meses** | Solo Sheets | 50K filas ralentizan todo |
| **Mes 4** | Sheets + `QUERY()` | Consolidación toma 2+ horas |
| **Mes 5+** | Sheets + BigQuery | ✅ Sin límites |

**Solución implementada:**
- Pipeline automático diario: Sheets → BigQuery
- Consultas SQL unificando 2M+ transacciones
- Sheets como interfaz con datos actualizados
- Análisis de cohorte y CLV antes imposibles

**Resultados:**
- ⏱️ Tiempo de análisis: **4 horas → 5 minutos**
- 📊 Capacidad: **50K → 2M+ filas**
- 💰 ROI: **+15% en ingresos** (oportunidades de upsell)
- 🎯 Precisión: Discrepancias **del 12% al 1%**

---

## 7. Configuración práctica

### Conectar Sheets → BigQuery:
1. En Sheets: *Datos → Conectores → BigQuery*
2. Seleccionar proyecto, dataset y tabla
3. Configurar actualización (horaria, diaria)

### Conectar BigQuery → Sheets:
1. En BigQuery: Crear tabla externa desde Sheets
2. Especificar URL y rango
3. Definir schema automático o manual

### Optimización de costos en BigQuery:
- ✅ Particionar tablas por fecha
- ✅ Usar `SELECT` explícito (no `SELECT *`)
- ✅ Aprovechar cache de consultas
- ✅ Programar queries pesadas en horas valle

**Ejemplo de ahorro:** Una empresa optimizó consultas y redujo costos mensuales de **$800 a $220**.

---

## 8. Integración con el ecosistema Google

| Herramienta | Rol | Valor |
|-------------|-----|-------|
| **Looker Studio** | Visualización | Dashboards profesionales |
| **Apps Script** | Automatización | Flujos personalizados |
| **Cloud Functions** | Serverless | Procesamiento event-driven |
| **Cloud Scheduler** | Orquestación | Ejecución programada |

**Arquitectura de referencia:**
```
Forms/APIs → Sheets → BigQuery → Looker Studio → Sheets (dashboards)
```

---

## 9. Resumen

**Google Sheets es ideal para:**
- ✅ Análisis colaborativos en tiempo real
- ✅ Prototipado rápido de dashboards
- ✅ Datos operativos livianos (<100K filas)

**BigQuery es esencial para:**
- ✅ Datasets masivos (millones+ de filas)
- ✅ Procesamiento histórico completo
- ✅ Machine Learning y análisis predictivo
- ✅ Performance crítico y escala

**La combinación gana cuando:**
- ✅ Necesitas agilidad con potencia
- ✅ Tienes stakeholders no técnicos
- ✅ Estás escalando gradualmente
- ✅ Quieres democratizar datos sin sacrificar capacidad

> **Clave estratégica:** No las veas como competencia, sino como compañeras en diferentes etapas del journey de datos.

---

## 10. Referencias

### Vídeos
- [Google BigQuery Tutorial](https://youtu.be/MH5M2Crn6Ag?si=2tqKyA9ERmRfkjKe)
- [Quick Data Analysis with Google Sheets](https://youtu.be/Y8jhi_yZKOg?si=iV4lDAddVxxygUMw)
- [Connect Google Sheets & BigQuery - Connected Sheets](https://youtu.be/sfgaZvAmcLs?si=bEWZP5h9VrR_shrO)

### Lecturas
- [Apps Script Documentation](https://developers.google.com/apps-script)
- [BigQuery SQL Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql)
- [Using Connected Sheets | BigQuery](https://cloud.google.com/bigquery/docs/connected-sheets)
